In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banque-sonore-data/Audio/utt_00946.wav
/kaggle/input/banque-sonore-data/Audio/utt_02806.wav
/kaggle/input/banque-sonore-data/Audio/utt_00472.wav
/kaggle/input/banque-sonore-data/Audio/utt_05266.wav
/kaggle/input/banque-sonore-data/Audio/utt_05171.wav
/kaggle/input/banque-sonore-data/Audio/utt_02248.wav
/kaggle/input/banque-sonore-data/Audio/utt_02988.wav
/kaggle/input/banque-sonore-data/Audio/utt_06666.wav
/kaggle/input/banque-sonore-data/Audio/utt_02938.wav
/kaggle/input/banque-sonore-data/Audio/utt_04330.wav
/kaggle/input/banque-sonore-data/Audio/utt_02842.wav
/kaggle/input/banque-sonore-data/Audio/utt_07170.wav
/kaggle/input/banque-sonore-data/Audio/utt_04092.wav
/kaggle/input/banque-sonore-data/Audio/utt_07114.wav
/kaggle/input/banque-sonore-data/Audio/utt_07267.wav
/kaggle/input/banque-sonore-data/Audio/utt_02750.wav
/kaggle/input/banque-sonore-data/Audio/utt_01732.wav
/kaggle/input/banque-sonore-data/Audio/utt_02622.wav
/kaggle/input/banque-sonore-data/Audio/utt_022

In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.0 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.

In [70]:
import sys

sys.path.append('/kaggle/input/python-code/text')

import os
import wave
import json
import jiwer
import audio



import whisper

In [71]:
import pandas as pd
df = pd.read_parquet("hf://datasets/Bretagne/Banque_Sonore_Dialectes_Bretons/train.parquet")

In [73]:
def fix_row(row):
    if 'peurunvan :' in row['fr']:
        # Extract Breton text after "peurunvan :"
        new_text = row['fr'].split('peurunvan :', 1)[1].strip()
        # Replace both Br and Fr columns with this new_text
        row['br'] = new_text
        row['fr'] = new_text
    return row

df = df.apply(fix_row, axis=1)

In [74]:
df

,audio,br,fr,city
0,{'bytes': b'RIFF$x\x06\x00WAVEfmt \x10\x00\x00...,Pell zo n'eo ket o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
1,{'bytes': b'RIFF$\xb1\x03\x00WAVEfmt \x10\x00\...,N'oc'h ket bet pell zo o welet ac'hanon.,Vous n'êtes pas venus me voir depuis longtemps.,22107
2,{'bytes': b'RIFF$r\x03\x00WAVEfmt \x10\x00\x00...,Pell zo n'eo ket bet o welet ac'hanon.,Ça fait longtemps qu'il n'est pas venu me voir.,22107
3,{'bytes': b'RIFF$\x97\x05\x00WAVEfmt \x10\x00\...,Ma... ma zi ne vo ket gwerzhet.,Ma... ma maison ne sera pas vendue.,22107
4,{'bytes': b'RIFF$\xa0\x05\x00WAVEfmt \x10\x00\...,Hon... hon ti ne vo ket gwerzhet.,Notre... notre maison ne sera pas vendue.,22107
...,...,...,...,...
7286,{'bytes': b'RIFF$\xce\x07\x00WAVEfmt \x10\x00\...,Ne zeue ket ag ar vro Kemper. Ne zeue ket ag a...,Il ne venait pas du pays de Quimper. Il ne ven...,56247
7287,{'bytes': b'RIFF$\xa6\x08\x00WAVEfmt \x10\x00\...,Bout a oa linad razh d'en-dro ar feunteun. Bou...,Il y avait des orties tout autour de la fontai...,56247
7288,{'bytes': b'RIFF$i\x03\x00WAVEfmt \x10\x00\x00...,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247
7289,{'bytes': b'RIFF$3\x03\x00WAVEfmt \x10\x00\x00...,Ne oa ket aes o zennañ.,Ce n'était pas facile de les arracher.,56247


In [75]:
dataset_path = "/kaggle/input/banque-sonore-data/Audio"

In [76]:
model = whisper.load_model("turbo")

100%|█████████████████████████████████████| 1.51G/1.51G [00:22<00:00, 72.8MiB/s]


In [77]:
from load_ground_truth import load_ground_truth_dict

In [78]:
import sys
sys.path.append('/kaggle/input/python-code/text')

from transcriber_whisper import transcribe_audio

In [13]:
!pip install universal_edit_distance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 6.4 MB/s eta 0:00:00a 0:00:01


In [79]:
sys.path.append('/kaggle/input/python-code/text')

from evaluate_wer_cer import evaluate_wer_cer

In [80]:
def generate_transcription(row):
    file_id = row['file']
    utt_name = f"utt_{file_id:05d}.wav"
    full_path = os.path.join(dataset_path, utt_name)
    return transcribe_audio(full_path, model)

In [81]:

small_df = df.sample(n=2000, random_state=42)  # random_state makes it reproducible
small_df

,audio,br,fr,city
1057,{'bytes': b'RIFF\x8c\xdf\x00\x00WAVEfmt \x10\x...,ur c'harzh,une haie,29032
2654,{'bytes': b'RIFF\xae\xe5\x04\x00WAVEfmt \x10\x...,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168
4401,{'bytes': b'RIFF$1\x08\x00WAVEfmt \x10\x00\x00...,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230
5843,{'bytes': b'RIFF\xa4\x81\x03\x00WAVEfmt \x10\x...,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293
7177,{'bytes': b'RIFF$e\x04\x00WAVEfmt \x10\x00\x00...,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175
...,...,...,...,...
981,{'bytes': b'RIFF8@\x01\x00WAVEfmt \x10\x00\x00...,skarzhañ,vider,29032
1732,{'bytes': b'RIFF$\xc4\x05\x00WAVEfmt \x10\x00\...,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150
6185,{'bytes': b'RIFF$F\x05\x00WAVEfmt \x10\x00\x00...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076
2354,{'bytes': b'RIFF\x08j\x06\x00WAVEfmt \x10\x00\...,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153


In [82]:
# Step 1: Filter where city starts with '22'
# df_22 = df[df['city'].astype(str).str.startswith('22')].copy()

# Step 2: Add a 'file' column with the original index
small_df['file'] = small_df.index

small_df['transcription'] = small_df.apply(generate_transcription, axis=1)

# Optional: Reset index for a clean new DataFrame
small_df = small_df.reset_index(drop=True)

Transcribing /kaggle/input/banque-sonore-data/Audio/utt_01057.wav...
[00:00.000 --> 00:00.360]  Gess.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_02654.wav...
[00:00.000 --> 00:02.800]  Mori, azeudu benne ne internoz.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_04401.wav...
[00:00.000 --> 00:05.720]  N'a ivez ur du eichel, loa, wachit, wachit memez mood.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_05843.wav...
[00:00.000 --> 00:02.000]  piu zubetitro e huine.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_07177.wav...
[00:00.000 --> 00:03.240]  a bei eem rwir Rolli, Chut
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_03805.wav...
[00:00.000 --> 00:02.340]  Es scho e'n tu mei'n besur.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_03961.wav...
[00:00.000 --> 00:02.400]  Ben mis laboureth mis gongiom et fée.
Transcribing /kaggle/input/banque-sonore-data/Audio/utt_06147.wav...
[00:00.000 --> 00:03.260]  Na' me zo bete, do'r ve

In [83]:
small_df

,audio,br,fr,city,file,transcription
0,{'bytes': b'RIFF\x8c\xdf\x00\x00WAVEfmt \x10\x...,ur c'harzh,une haie,29032,1057,Gess.
1,{'bytes': b'RIFF\xae\xe5\x04\x00WAVEfmt \x10\x...,Mari zo deuet a-benn enno antronoz.,Marie est venue vers eux le lendemain.,29168,2654,"Mori, azeudu benne ne internoz."
2,{'bytes': b'RIFF$1\x08\x00WAVEfmt \x10\x00\x00...,"An amzer gwechall, ne oa... ne oa ket, eu... n...","Autrefois, c'était... c'était, euh... ça n'éta...",29230,4401,"N'a ivez ur du eichel, loa, wachit, wachit me..."
3,{'bytes': b'RIFF\xa4\x81\x03\x00WAVEfmt \x10\x...,Piv zo bet o troc'hiñ ar wezenn ?,Qui a coupé l'arbre ?,29293,5843,piu zubetitro e huine.
4,{'bytes': b'RIFF$e\x04\x00WAVEfmt \x10\x00\x00...,"Pa oufemp ar wirionez, n'he lavarfemp ket.","Si nous savions la vérité, nous ne la dirions ...",56175,7177,"a bei eem rwir Rolli, Chut"
...,...,...,...,...,...,...
1995,{'bytes': b'RIFF8@\x01\x00WAVEfmt \x10\x00\x00...,skarzhañ,vider,29032,981,hati gä sche!
1996,{'bytes': b'RIFF$\xc4\x05\x00WAVEfmt \x10\x00\...,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,Mets tes mains dans ta poche ! Retire-les d'en...,29150,1732,lak手'zh urnu bat abouš tenne mez tusta avoll!
1997,{'bytes': b'RIFF$F\x05\x00WAVEfmt \x10\x00\x00...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...","Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",56076,6185,D'ai ca nem brema reperdaiznia.ENTSA O MIMI M...
1998,{'bytes': b'RIFF\x08j\x06\x00WAVEfmt \x10\x00\...,"Oh nann, mes me a gred lar mar vije bet chomet...","Oh nann, mes me a gred lar mar vije bet chomet...",29153,2354,tan ke do thuom


In [84]:
from filter_char import filter_out_chars
from normalizer import normalize_sentence
from utils import pre_process


def process_br_text(br):
    PUNCTUATION = '<>.?!,;:«»“”"()[]/…–—•'
    br = filter_out_chars(br, PUNCTUATION + '*')
    br = normalize_sentence(br, autocorrect=True)
    br = pre_process(br).replace('-', ' ').lower()
    return br

In [85]:
small_df['br_processed'] = small_df['br'].apply(process_br_text)

In [86]:
from jiwer import wer, cer

# Compute WER and CER for each row
small_df['wer'] = small_df.apply(lambda row: wer(row['br_processed'], row['transcription']), axis=1)
small_df['cer'] = small_df.apply(lambda row: cer(row['br_processed'], row['transcription']), axis=1)

In [87]:
small_df = small_df[['br', 'br_processed', 'fr', 'transcription', 'city', 'wer', 'cer', 'file']]
small_df

,br,br_processed,fr,transcription,city,wer,cer,file
0,ur c'harzh,ur c'harzh,une haie,Gess.,29032,1.0,1.000000,1057
1,Mari zo deuet a-benn enno antronoz.,mari zo deuet a benn enno antronoz,Marie est venue vers eux le lendemain.,"Mori, azeudu benne ne internoz.",29168,1.0,0.529412,2654
2,"An amzer gwechall, ne oa... ne oa ket, eu... n...",an amzer gwechall ne oa ne oa ket eu ne oa ket...,"Autrefois, c'était... c'était, euh... ça n'éta...","N'a ivez ur du eichel, loa, wachit, wachit me...",29230,1.0,0.660714,4401
3,Piv zo bet o troc'hiñ ar wezenn ?,piv zo bet o troc'hiñ ar wezenn,Qui a coupé l'arbre ?,piu zubetitro e huine.,29293,1.0,0.612903,5843
4,"Pa oufemp ar wirionez, n'he lavarfemp ket.",pa oufemp ar wirionez n'he lavarfemp ket,"Si nous savions la vérité, nous ne la dirions ...","a bei eem rwir Rolli, Chut",56175,1.0,0.750000,7177
...,...,...,...,...,...,...,...,...
1995,skarzhañ,skarzhañ,vider,hati gä sche!,29032,3.0,1.500000,981
1996,Lak da zornoù e-barzh da boch ! Tenn anezhe er...,lak da zornoù e barzh da boch tenn anezhe er m...,Mets tes mains dans ta poche ! Retire-les d'en...,lak手'zh urnu bat abouš tenne mez tusta avoll!,29150,1.0,0.606061,1732
1997,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",nend nend ay ket ganin bremañ c'hwi me am eus ...,"Nend, nend ay ket ganin bremañ, c'hwi [?], me ...",D'ai ca nem brema reperdaiznia.ENTSA O MIMI M...,56076,1.0,0.804878,6185
1998,"Oh nann, mes me a gred lar mar vije bet chomet...",oh nann mes me a gred lâr mar vije bet chomet ...,"Oh nann, mes me a gred lar mar vije bet chomet...",tan ke do thuom,29153,1.0,0.875000,2354


In [89]:
city_avg = small_df.groupby('city')[['wer', 'cer']].mean()

# Optional: round to 2 decimal places
city_avg = city_avg.round(2)

for city_code, row in city_avg.iterrows():
    print(f"Average WER and CER for {city_code}:\nWER: {row['wer']}\nCER: {row['cer']}\n")

Average WER and CER for 22107:
WER: 1.0
CER: 0.68

Average WER and CER for 22167:
WER: 1.01
CER: 0.63

Average WER and CER for 22181:
WER: 0.99
CER: 0.72

Average WER and CER for 22230:
WER: 1.01
CER: 0.66

Average WER and CER for 22331:
WER: 0.98
CER: 0.59

Average WER and CER for 22336:
WER: 0.98
CER: 0.56

Average WER and CER for 22386:
WER: 0.96
CER: 0.65

Average WER and CER for 29005:
WER: 1.01
CER: 0.68

Average WER and CER for 29031:
WER: 1.75
CER: 1.45

Average WER and CER for 29032:
WER: 1.05
CER: 0.72

Average WER and CER for 29058:
WER: 0.91
CER: 0.54

Average WER and CER for 29080:
WER: 1.06
CER: 0.67

Average WER and CER for 29082:
WER: 1.02
CER: 0.77

Average WER and CER for 29091:
WER: 0.97
CER: 0.61

Average WER and CER for 29105:
WER: 0.98
CER: 0.47

Average WER and CER for 29122:
WER: 0.98
CER: 0.71

Average WER and CER for 29136:
WER: 1.05
CER: 0.71

Average WER and CER for 29139:
WER: 0.98
CER: 0.71

Average WER and CER for 29146:
WER: 0.98
CER: 0.67

Average WER a

In [90]:
# Filter for cities starting with '22'
small_df_22 = small_df[small_df['city'].str.startswith('22')]
avg_22 = small_df_22[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '22'
small_df_29 = small_df[small_df['city'].str.startswith('29')]
avg_29 = small_df_29[['wer', 'cer']].mean().round(4)


# Filter for cities starting with '56'
small_df_56 = small_df[small_df['city'].str.startswith('56')]
avg_56 = small_df_56[['wer', 'cer']].mean().round(4)



print("Evaluation for --Côtes-d'Armor-- dialects 22xxx :")
print(f"WER: {avg_22['wer']}")
print(f"CER: {avg_22['cer']}\n")

print("Evaluation for --Finistère-- dialects 29xxx :")
print(f"WER: {avg_29['wer']}")
print(f"CER: {avg_29['cer']}\n")

print("Evaluation for --Morbihan-- dialects 56xxx :")
print(f"WER: {avg_56['wer']}")
print(f"CER: {avg_56['cer']}")

Evaluation for --Côtes-d'Armor-- dialects 22xxx :
WER: 1.0001
CER: 0.6469

Evaluation for --Finistère-- dialects 29xxx :
WER: 1.0132
CER: 0.6881

Evaluation for --Morbihan-- dialects 56xxx :
WER: 1.0142
CER: 0.7218


In [91]:
average_wer = small_df['wer'].mean()
average_cer = small_df['cer'].mean()

# Optional: round them
average_wer = round(average_wer, 2)
average_cer = round(average_cer, 2)

print(f"Overall Average WER: {average_wer}")
print(f"Overall Average CER: {average_cer}")

Overall Average WER: 1.01
Overall Average CER: 0.69
